In [1]:
# format to specify is the one from thermonet
# needs to bee: P03958A 263 W F
# ie: uniprot+mutated_chain mutation_position wild_aa mutated_aa

In [2]:
import pandas as pd
import numpy as np
import os
from glob import glob

In [3]:
DATASET_INPUT = "../data/main_dataset_creation/outputs/all_v2_2/dataset_with_alphafold_paths.csv"
COMPUTE_NEW_SCORES = True
CLEAN_MUTATIONS = True
START_FRESH = False

ROSETTA_BIN_DIR_PARIS = "/home/ml/novozymes-prediction/resources/rosetta/rosetta_bin_linux_2021.16.61629_bundle/main/source/bin/"
ROSETTA_BIN_DIR_SOUTH = "/home/tom/Documents/Kaggle/novozymes-prediction/resources/rosetta_bin_linux_2021.16.61629_bundle/main/source/bin/"
SCORE_BIN_PARIS = f"{ROSETTA_BIN_DIR_PARIS}score_jd2.static.linuxgccrelease"
SCORE_BIN_SOUTH = f"{ROSETTA_BIN_DIR_SOUTH}score_jd2.static.linuxgccrelease"
THREADS_PARIS = 6
THREADS_SOUTH = 0
THREADS_GCP = 0
PARIS_RATIO = 1
GCP_RATIO = 0
THREADS = THREADS_PARIS+THREADS_SOUTH

SUBSET_DUPLICATES_NO_PH = ["uniprot", "wild_aa", "mutation_position",
                           "mutated_aa", "sequence"]

MAX_CONSECUTIVE_TASKS = 40


In [4]:
df = pd.read_csv(DATASET_INPUT)
df = df[~(df.alphafold_path.isna())]
df = df[~(df.ddG.isna())]
df.drop_duplicates(subset=SUBSET_DUPLICATES_NO_PH, inplace=True)
df["already_computed"] = False
print(len(df))

5704


In [5]:
print(len(df.uniprot.unique()))
print(len(df.alphafold_path.unique()))

297
294


In [6]:
all_3D_structures = glob("./relaxed_pdb/**/*.pdb")
all_3D_structures += glob("../data/main_dataset_creation/3D_structures/alphafold/*.pdb")
print(len(all_3D_structures))

9760


In [7]:
if START_FRESH:
    already_computed = []
else:
    already_computed = glob("./relaxed_pdb/**/*.sc")
    already_computed += glob(
        "../data/main_dataset_creation/3D_structures/alphafold/*.sc")

print(len(already_computed))
print(already_computed[:2])

9570
['./relaxed_pdb/P51677_relaxed/P51677_relaxed.sc', './relaxed_pdb/Q93P54_relaxed/Q93P54_relaxed.sc']


In [8]:
crashed = []
# crashed = glob("relaxed_pdb/**/*CRASH*")
# crashed = [n.split('/')[-2] for n in crashed]
# print(len(crashed))
# print(crashed[:5])


In [9]:
# split all_3D_structures between CPUs
PARIS_index = int(len(all_3D_structures)*PARIS_RATIO)
GCP_index = PARIS_index+int(len(all_3D_structures)*GCP_RATIO)
structure_PARIS = all_3D_structures[:PARIS_index]
structure_GCP = all_3D_structures[PARIS_index:GCP_index]
structure_SOUTH = all_3D_structures[GCP_index:]
print(PARIS_index, GCP_index, len(all_3D_structures))


9760 9760 9760


In [10]:
# create bash scripts

to_compute = 0
for i, structure_path in enumerate(structure_PARIS):
    script_suffix = f"PARIS_{i % THREADS_PARIS}"
    name, _ = os.path.splitext(structure_path.split("/")[-1])
    dir_path = '/'.join(structure_path.split("/")[:-1])
    # print(f"{dir_path}/{name}.sc")
    if f"{dir_path}/{name}.sc" in already_computed:
        continue
    if name in crashed:
        continue
    with open(f"scores_{script_suffix}.sh", "a+") as f:
        cmd = f"{SCORE_BIN_PARIS} -in:file:s {structure_path} -out:file:score_only -scorefile_format json -out:file:scorefile {dir_path}/{name}.sc > score_logs/{name}.log"
        f.write(cmd)
        f.write("\n")
        to_compute += 1

print(to_compute)


522


In [11]:
with open("main_scores_PARIS.sh", "w+") as f:
    for i in range(THREADS_PARIS):
        f.write(f"bash scores_PARIS_{i}.sh & \n")

# with open("main_mutations_SOUTH.sh", "w+") as f:
#     for i in range(THREADS_SOUTH):
#         f.write(f"bash mutations_SOUTH_{i}.sh & \n")
